In [1]:
import sys
sys.path.insert(1, '../../../..')

In [3]:
import optuna
from optuna.trial import TrialState
from optuna.samplers import TPESampler

import torch
import torch.optim as optim
import torch.nn as nn

import numpy as np
import sklearn
import copy

from methods.PINN import PINN
from methods.methodsDataset.MLPINNDataset import MLPINNDataset
from methods.DataDrivenMethods import DDMethod
from solvers.Solver import Solver

In [4]:
params_solver = {'equation': 'Poisson', 'domain': [0., 1.], 'D': 1., 'nx': 101}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
params_MLPINN = {'layer_dims': None, 'activations': None, 'device': device, 'seed': 123}

In [5]:
Dmin, Dmax = 0.1, 10
D_list = np.linspace(Dmin, Dmax, 1000)

U_sols = []
solver = Solver(params={'solver': params_solver})
for d in D_list:
    solver.change_D(new_D=d)
    U_sols.append(solver.solve())

U_sols = np.stack(U_sols)

In [7]:
d_train, d_val, u_train, u_val = sklearn.model_selection.train_test_split(D_list, U_sols, test_size=0.2, random_state=123)

D_train_repeated = torch.Tensor([[d] * len(solver.x) for d in d_train]).view(-1, 1)
D_val_repeated = torch.Tensor([[d] * len(solver.x) for d in d_val]).view(-1, 1)
x = torch.Tensor(solver.x).view(-1, 1)
X_train = x.repeat(d_train.shape[0], 1)
X_val = x.repeat(d_val.shape[0], 1)

DX_train = torch.cat((D_train_repeated, X_train), dim=1)
DX_val = torch.cat((D_val_repeated, X_val), dim=1)
dU_train = torch.Tensor(u_train.flatten())
dU_val = torch.Tensor(u_val.flatten())

In [8]:
def get_dataset():
    return DX_train, DX_val, dU_train, dU_val

In [9]:
def loss_fn(x, y = 0):
    return torch.square(y - x).mean()

In [14]:
def define_model(trial, input_size, output_size):
    n_layers = trial.suggest_int("n_layers", 1, 5)
    layers = [input_size]
    activations = []
    for i in range(n_layers):
        out_features = trial.suggest_int("units_l{}".format(i), 32, 128)
        activation = trial.suggest_categorical("activation_l{}".format(i), ["tanh", "softplus", "relu"])
        layers += [out_features]
        activations.append(activation)
    layers += [output_size]
    params_MLPINN_trial = copy.deepcopy(params_MLPINN)
    params_MLPINN_trial['layer_dims'] = layers
    params_MLPINN_trial['activations'] = activations
    return PINN(params={'solver':None, 'method':params_MLPINN_trial})

In [21]:
def objective(trial):
    # Generate the model.
    model = define_model(trial, 2, 1)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    # optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    
    DX_train, DX_val, U_train, U_val = get_dataset()

    trainDataset = MLPINNDataset(x=DX_train, y=U_train)
    valDataset = MLPINNDataset(x=DX_val, y=U_val)

    batch_size = trial.suggest_int("batch_size", 20000, 40000)
    torch.manual_seed(params_MLPINN['seed'])
    # trainLoader = torch.utils.data.DataLoader(trainDataset, batch_size=batch_size, shuffle=True)
    # valLoader = torch.utils.data.DataLoader(valDataset, batch_size=batch_size, shuffle=False)

    hyperparameters = {'lr': lr, 'epochs': 5000, 'optimizer': optimizer_name, 'batch_size': batch_size}
    # Training of the model.
    model.fit_supervised(hyperparameters = hyperparameters, DX_train = DX_train, DX_val = DX_val, U_train=U_train, U_val=U_val)
    
    val_loss = model.loss_dict['val']
    val_tot = [sum(x) for x in zip(val_loss['ic_bc'], val_loss['residual'], val_loss['data_driven'])]
    trial.report(min(val_tot), 5000)

    # Handle pruning based on the intermediate value.
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()

    return min(val_tot)

In [22]:
sampler = TPESampler(seed=params_MLPINN['seed'])
study = optuna.create_study(direction="minimize", sampler=sampler)

[I 2023-08-07 10:23:09,382] A new study created in memory with name: no-name-8c8c2449-327a-4ef4-b29a-c8c47e7cd1bd


In [23]:
study.optimize(objective, n_trials=10)

[epoch: 10 : 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [01:02<00:00,  5.68s/it]
[I 2023-08-07 10:24:13,419] Trial 0 finished with value: 0.2199098595204608 and parameters: {'n_layers': 4, 'units_l0': 59, 'activation_l0': 'relu', 'units_l1': 73, 'activation_l1': 'tanh', 'units_l2': 70, 'activation_l2': 'softplus', 'units_l3': 37, 'activation_l3': 'softplus', 'optimizer': 'SGD', 'lr': 0.003448290408913141, 'batch_size': 3699}. Best is trial 0 with value: 0.2199098595204608.
[epoch: 10 : 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:45<00:00,  4.11s/it]
[I 2023-08-07 10:24:59,848] Trial 1 finished with value: 0.12677240412449464 and parameters: {'n_layers': 4, 'units_l0': 91, 'activation_

In [ ]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
activations = []
layer_dims = []

for key, value in trial.params.items():
    if key.split('_')[0] == 'activation':
        activations.append(value)
    elif key.split('_')[0] == 'units':
        layer_dims.append(value)
    elif key == 'optimizer':
        optimizer = value
    elif key == 'lr':
        lr = value

Use this to fill config_step_1.py

In [ ]:
print(activations, '\n', layer_dims, '\n', optimizer, '\n', lr)